<a href="https://colab.research.google.com/github/Atharva696969/skill-eclipse/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install opencv-python-headless
!pip install matplotlib

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16941, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 16941 (delta 70), reused 92 (delta 48), pack-reused 16805 (from 1)
Receiving objects: 100% (16941/16941), 15.70 MiB | 8.29 MiB/s, done.
Resolving deltas: 100% (11603/11603), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
import cv2
import torch
from google.colab.patches import cv2_imshow # Import the cv2_imshow function

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Initialize video capture (0 for webcam or provide video file path)
cap = cv2.VideoCapture('/content/traffic-mini.mp4')

# Check if the video capture is opened successfully
if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

# Create a tracker
tracker = cv2.TrackerCSRT_create()

# Initialize variables
init_once = False
bbox = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)

    # Get bounding boxes and labels
    detections = results.xyxy[0]  # Get detections for the first image
    for *xyxy, conf, cls in detections:
        label = f'{model.names[int(cls)]} {conf:.2f}'
        bbox = [int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])]

        # Initialize tracker on the first frame
        if not init_once:
            tracker.init(frame, bbox)
            init_once = True

        # Draw the bounding box and label on the frame
        cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 0, 0), 2)
        cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Update tracker
    if init_once and bbox is not None:
        success, bbox = tracker.update(frame)
        if success:
            # Draw the tracked bounding box
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (0, 255, 0), 2)

    # Display the resulting frame
    cv2_imshow(frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()